In [1]:
import tensorflow as tf
from tensorflow import keras

from time import time
from tensorflow.python.keras.callbacks import TensorBoard

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler

import pickle
from sklearn import metrics

print(tf.__version__)

2.0.0-beta0


In [2]:
PATH = os.getcwd()

In [4]:
X_train = pd.read_csv(PATH + "/Train/X_train.csv", header=0)
Y_train = pd.read_csv(PATH + "/Train/Y_train.csv", header=0)

X_test = pd.read_csv(PATH + "/Test/X_test.csv", header=0)
Y_test = pd.read_csv(PATH + "/Test/Y_test.csv", header=0)

In [5]:
def ensemble_prediction(predictions):
    after = []
    #final_pred = np.zeros(3)
    for i in range(0,len(X_test)):
        if np.array_equal(predictions[0][i],predictions[1][i]):
            mo = predictions[0][i]
            after.append(list(mo))
            continue
        if np.array_equal(predictions[0][i],predictions[2][i]):
            mo = predictions[0][i]
            after.append(list(mo))
            continue
        if np.array_equal(predictions[1][i],predictions[2][i]):
            mo = predictions[1][i]
            after.append(list(mo))
            continue
        after.append(list(mo))
    out = np.asarray(after)
    return out

In [7]:
with open(PATH + '/Random_Forest/rf3.cpickle', 'rb') as f:
    clf = pickle.load(f)

In [15]:
model1 = keras.models.load_model(PATH + '/NN_models/3layers_mlp.h5')
model2 = keras.models.load_model(PATH + '/NN_models/5layers_residual.h5')
model3 = clf

pred1=model1.predict(X_test)
pred1 = tf.one_hot(tf.argmax(pred1, axis=1), depth = 3)
pred2=model2.predict(X_test)
pred2 = tf.one_hot(tf.argmax(pred2, axis=1), depth = 3)
pred3=model3.predict(X_test)

#print(pred2)

predictions = [pred1, pred2, pred3]
print(pred1.shape)

(3950, 3)


In [9]:
print(pred2.shape)
print(pred3.shape)

(3950, 3)
(3950, 3)


In [19]:
Y_ensemble = ensemble_prediction(predictions)
print(Y_ensemble.shape)
print("Accuracy:",metrics.accuracy_score(Y_test, pred1))

(3950, 3)
Accuracy: 0.7610126582278481
